## MNIST 使用
MNIST is a computer vision dataset. It consists of black and white images from zero to nine. Each image is 28 * 28 and have been flatten to 784 dimension vector. Also, it includes labels for each image, telling us which digit it is.

![Alt text](./images/dnn_implement/Selection_017.png)
![Alt text](./images/dnn_implement/Selection_018.png)


The MNIST data is split into three parts: 
1. 55000 training data (mnist.train) with a shape of [55000, 784]
2. 10000 test data (mnist.test) with a shape of [10000, 784]
3. 5000 validation data (mnist.validation) with a shape of [5000, 784]

you can access:  
training image as `mnist.train.images` (see below picture)  
training label as `mnist.train.labels` (see below picture)  
test image as `mnist.test.images`   
test label as `mnist.test.labels`   

Note that label is encoded as "one-hot vectors", which mean if the target image is 2, the label should be [0,0,1,0,0,0,0,0,0,0]


![Alt text](./images/dnn_implement/Selection_021.png)
![Alt text](./images/dnn_implement/Selection_020.png)

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.reset_default_graph()

# Load mnist dataset
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

batch_xs, batch_ys = mnist.train.next_batch(5)

print('first 5 labels {}:\n'.format(batch_ys))
print('first 5 labels shape {}:\n'.format(batch_ys.shape))

print('first 5 images {}:\n'.format(batch_xs))
print('first 5 labels shape {}:\n'.format(batch_ys.shape))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
first 5 labels [[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]:

first 5 labels shape (5, 10):

first 5 images [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0

# 建立簡單模型

In [7]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.reset_default_graph()

# Load mnist dataset
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# Define image input 784 = 28 * 28. Note that DNN input is a vector
# [None, 784] mean that there are a batch of data and each of them is 784 dimension vector
x = tf.placeholder(tf.float32, [None, 784])

# Define label. There are totally 10 class (0-9)
y = tf.placeholder(tf.float32, [None, 10])

W = tf.Variable(tf.truncated_normal([784, 10], stddev=0.1))
b = tf.Variable(tf.truncated_normal([10], stddev=0.1))
y_predict = tf.matmul(x, W) + b


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    batch_xs, batch_ys = mnist.train.next_batch(5)
        
    y_predict_ = sess.run(y_predict, feed_dict={x: batch_xs, y: batch_ys}) # 此處的Y: batch_ys 似乎沒用到, 與y_predict_作比較
    print(y_predict_)
    print(y_predict_.shape)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
[[-5.4069692e-01 -5.6583248e-03 -4.5042193e-01  2.3590329e-01
  -3.9751977e-03 -8.6558096e-02 -4.9344443e-02  2.7962282e-01
   7.2098166e-01  1.1871649e-01]
 [-7.6714158e-03 -1.0220045e+00 -8.9830381e-01 -2.6027873e-01
   4.5678353e-01  1.1773648e+00 -1.7488407e+00  3.0439454e-01
  -3.6758482e-01  5.1784229e-01]
 [-4.8161945e-01 -1.3847558e+00 -6.9110370e-01 -1.4456996e+00
   1.4518343e+00 -2.1155497e-01 -6.5869248e-01 -1.2537712e+00
   1.0142154e+00 -1.0557004e+00]
 [-6.8580508e-01 -3.0447328e-01 -7.1959026e-02 -2.5982603e-01
   5.7477522e-01  8.0432117e-02 -2.0448026e-01 -7.4660921e-01
  -1.7064403e-01 -1.0040458e+00]
 [ 1.3591874e+00 -5.1231062e-01  2.6188046e-04  4.3590283e-01
   1.8229458e+00  1.2047427e+00 -1.7926030e+00 -1.9817132e-01
   1.1507736e+00 -6.6125560e-01]]
(5, 10)


## 建立DNN神經網路模型 

In [20]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.reset_default_graph()

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

INPUT_NODE =784

LAYER1_NODE = 128
LAYER2_NODE = 64
LAYER3_NODE = 10


x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

W1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
b1 = tf.Variable(tf.truncated_normal([LAYER1_NODE], stddev=0.1))
W2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, LAYER2_NODE], stddev=0.1))
b2 = tf.Variable(tf.truncated_normal([LAYER2_NODE], stddev=0.1))
W3 = tf.Variable(tf.truncated_normal([LAYER2_NODE, LAYER3_NODE], stddev=0.1))
b3 = tf.Variable(tf.truncated_normal([LAYER3_NODE], stddev=0.1))

layer_1 = tf.matmul(x, W1) + b1
out1 = tf.nn.relu(layer_1)
layer_2 = tf.matmul(out1, W2) + b2
out2 = tf.nn.relu(layer_2)
layer_3 = tf.matmul(out2, W3) + b3
out3 = tf.nn.relu(layer_3)

y_predict = out3


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    batch_xs, batch_ys = mnist.train.next_batch(5)
        
    y_predict_ = sess.run(y_predict, feed_dict={x: batch_xs, y: batch_ys})
    print(y_predict_)
    print(y_predict_.shape)
    

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
[[0.00731017 0.43204063 0.32522568 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.1467699  0.5691537  0.6157013  0.         0.         0.
  0.         0.         0.         0.        ]
 [0.262901   0.243057   0.21182129 0.04192616 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.5449987  0.23554099 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.20826446 0.         0.2627445  0.         0.
  0.         0.04342301 0.         0.        ]]
(5, 10)
